# By providing examples of extraction, get LLM to follow and extract info in JSON

## Full run

In [1]:
%cd ..
from src import *
%cd experiments/

def extract_code_from_url(url: str):
    last_question_mark = url.rfind('?')
    code = url[last_question_mark - 32:last_question_mark]
    return code

def pipeline(filename: str, pageID: str):
    """Runs full pipeline"""
    docIngest(filename).run()
    mdCombine(filename).run()
    getChunks(filename).run()
    Extractor(filename).run()
    Notion(filename, pageID).run()

pageURL = "https://zain273work.notion.site/Test3-11fa89672d99805fb667f5bfc1d9f563?pvs=4"
pageID = extract_code_from_url(pageURL)
# print(pageID)
from datetime import datetime
current_datetime = datetime.now()

# Format it as a string
formatted_datetime: str = current_datetime.strftime("%Y%m%d%H%M%S")
name: str = "experiment" + formatted_datetime
print(f"Running {name}")
pipeline(name, pageID)

c:\Users\Zain\Documents\Github\University\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\Zain\Documents\Github\University


c:\Users\Zain\Documents\Github\University\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\Zain\Documents\Github\University\experiments
Running experiment20241015100115
Processing chunk0.txt
Processing chunk1.txt
Processing chunk10.txt
Processing chunk11.txt
Processing chunk12.txt
Processing chunk13.txt
Processing chunk14.txt
Processing chunk15.txt
Processing chunk16.txt
Processing chunk17.txt
Processing chunk18.txt
Processing chunk2.txt
Processing chunk3.txt
Processing chunk4.txt
Processing chunk5.txt
Processing chunk6.txt
Processing chunk7.txt
Processing chunk8.txt
Processing chunk9.txt


## Pip installs (if required)

In [ ]:
%pip install -r ../requirements.txt

## Imports

In [ ]:
%cd ..
from src import *
%cd experiments/

## Setup

In [2]:
def extract_code_from_url(url: str):
    last_question_mark = url.rfind('?')
    code = url[last_question_mark - 32:last_question_mark]
    return code

def pipeline(filename: str, pageID: str):
    """Runs full pipeline"""
    docIngest(filename).run()
    mdCombine(filename).run()
    getChunks(filename).run()
    Extractor(filename).run()
    # Squasher(filename).run()
    Notion(filename, pageID).run()

## Pipeline

In [ ]:
pageURL = "https://zain273work.notion.site/Test3-11fa89672d99805fb667f5bfc1d9f563?pvs=4"
pageID = extract_code_from_url(pageURL)
# print(pageID)
from datetime import datetime
current_datetime = datetime.now()

# Format it as a string
formatted_datetime: str = current_datetime.strftime("%Y%m%d%H%M%S")
name: str = "experiment" + formatted_datetime
print(f"Running {name}")
pipeline(name, pageID)

## Testing

In [ ]:
pageURL = "https://zain273work.notion.site/newTest-11fa89672d9980f2b846eb8e98c1c3e4?pvs=4"
pageID = extract_code_from_url(pageURL)
name = "experiment20241014200958"
Notion(name, pageID).run()